# US Name
Estimate sign_of_effect as a function of  publication_year and others variables


# Description

Estimate multi-probit

## Variables
### Target

sign_of_effect

### Features

- publication_year
- publication_type
- cnrs_ranking
- study_focused_on_social_environmental_behaviour
- type_of_data
- number_of_observations
- evaluation_method_of_the_link_between_csr_and_cfp
- developed_new
- econometric_method

## Complementary information



# Metadata

- Key: 176_esg_metadata
- Epic: Models
- US: Replicate previous results
- Task tag: #data-analysis, #meta-analysis, #mutlinomial-probit
- Analytics reports: 

# Input Cloud Storage

## Table/file

**Name**

- https://github.com/thomaspernet/esg_metadata/blob/master/01_data_preprocessing/00_download_data/ESG/esg_metaanalysis.py

**Github**

- papers_meta_analysis



# Connexion server

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_glue import service_glue
from pathlib import Path
import pandas as pd
import numpy as np
#import seaborn as sns
import os, shutil, json
import sys

path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)


name_credential = 'financial_dep_SO2_accessKeys.csv'
region = 'eu-west-3'
bucket = 'datalake-datascience'
path_cred = "{0}/creds/{1}".format(parent_path, name_credential)

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False)
glue = service_glue.connect_glue(client = client) 

In [3]:
pandas_setting = True
if pandas_setting:
    #cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'


# Load tables

Since we load the data as a Pandas DataFrame, we want to pass the `dtypes`. We load the schema from Glue to guess the types

In [5]:
db = 'esg'
table = 'papers_meta_analysis'

In [6]:
dtypes = {}
schema = (glue.get_table_information(database = db,
                           table = table)
          ['Table']['StorageDescriptor']['Columns']
         )
for key, value in enumerate(schema):
    if value['Type'] in ['varchar(12)',
                         'varchar(3)',
                        'varchar(14)', 'varchar(11)']:
        format_ = 'string'
    elif value['Type'] in ['decimal(21,5)', 'double', 'bigint', 'int', 'float']:
        format_ = 'float'
    else:
        format_ = value['Type'] 
    dtypes.update(
        {value['Name']:format_}
    )

In [7]:
download_data = True
filename = 'df_{}'.format(table)
full_path_filename = 'SQL_OUTPUT_ATHENA/CSV/{}.csv'.format(filename)
path_local = os.path.join(str(Path(path).parent.parent.parent), 
                              "00_data_catalog/temporary_local_data")
df_path = os.path.join(path_local, filename + '.csv')
if download_data:
    
    s3 = service_s3.connect_S3(client = client,
                          bucket = bucket, verbose = False)
    query = """
    SELECT * 
    FROM {}.{}
    """.format(db, table)
    try:
        df = (s3.run_query(
            query=query,
            database=db,
            s3_output='SQL_OUTPUT_ATHENA',
            filename=filename,  # Add filename to print dataframe
            destination_key='SQL_OUTPUT_ATHENA/CSV',  #Use it temporarily
            dtype = dtypes
        )
                )
    except:
        pass
    s3.download_file(
        key = full_path_filename
    )
    shutil.move(
        filename + '.csv',
        os.path.join(path_local, filename + '.csv')
    )
    s3.remove_file(full_path_filename)
    df.head()

In [8]:
pd.DataFrame(schema)

,Name,Type,Comment
0,nr,bigint,NaN
1,title,string,NaN
2,first_author,string,NaN
3,second_author,string,NaN
4,third_author,string,NaN
5,publication_year,bigint,NaN
6,publication_type,string,NaN
7,publication_name,string,NaN
8,cnrs_ranking,string,NaN
9,ranking,int,


## Schema Latex table

To rename a variable, please use the following template:

```
{
    'old':'XX',
    'new':'XX_1'
    }
```

if you need to pass a latex format with `\`, you need to duplicate it for instance, `\text` becomes `\\text:

```
{
    'old':'working\_capital\_i',
    'new':'\\text{working capital}_i'
    }
```

Then add it to the key `to_rename`

In [9]:
add_to_dic = True
if add_to_dic:
    if os.path.exists("schema_table.json"):
        os.remove("schema_table.json")
    data = {'to_rename':[], 'to_remove':[]}
    dic_rename = [
        {
        'old':'working\_capital\_i',
        'new':'\\text{working capital}_i'
        },
        {
        'old':'periodTRUE',
        'new':'\\text{period}'
        },
        {
        'old':'tso2\_mandate\_c',
        'new':'\\text{policy mandate}_'
        },
    ]

    data['to_rename'].extend(dic_rename)
    with open('schema_table.json', 'w') as outfile:
        json.dump(data, outfile)

In [10]:
sys.path.append(os.path.join(parent_path, 'utils'))
import latex.latex_beautify as lb
#%load_ext autoreload
#%autoreload 2

In [11]:
options(warn=-1)
library(tidyverse)
#library(lfe)
#library(lazyeval)
library(nnet)
library('progress')
path = "../../../utils/latex/table_golatex.R"
source(path)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



In [134]:
#library(mlogit)

In [72]:
%get df_path
df_final <- read_csv(df_path) %>%
mutate_if(is.character, as.factor) %>%
mutate(sign_of_effect = relevel(sign_of_effect, ref='Positive'),
      nr = as.factor(nr))
   # mutate_at(vars(starts_with("fe")), as.factor) %>%
#mutate(VAR_TO_RELEVEL = relevel(XX, ref='XX'))

Parsed with column specification:
cols(
  .default = col_character(),
  nr = col_double(),
  publication_year = col_double(),
  ranking = col_double(),
  first_date_of_observations = col_double(),
  last_date_of_observations = col_double(),
  number_of_observations = col_double(),
  level_of_significancy = col_double(),
  standard_error = col_double(),
  tstatistic_calculated_with_formula = col_double(),
  pvalue_calculated_with_formula = col_double(),
  effect_coeffient_estimator_beta = col_double(),
  adjusted_coefficient_of_determination = col_double()
)
See spec(...) for full column specifications.


## Table 1:Multinomial logit

$$
\begin{aligned}
\text{Write your equation}
\end{aligned}
$$

**Candidates**

- publication_year
- ~publication_type~
- ~cnrs_ranking~
- study_focused_on_social_environmental_behaviour
- ~type_of_data~
- ~number_of_observations~
- evaluation_method_of_the_link_between_csr_and_cfp
- developed_new
- ~econometric_method~

More about multinomial logit:

- https://dynalist.io/d/BP0wtNmNu-p8iF_bzH0OKX6i
- https://www.princeton.edu/~otorres/LogitR101.pdf

In [16]:
df_final%>%head(2)

nr,title,first_author,second_author,third_author,publication_year,publication_type,publication_name,cnrs_ranking,ranking,...,comments,cfp_regrouping,level_of_significancy,sign_of_effect,standard_error,tstatistic_calculated_with_formula,pvalue_calculated_with_formula,effect_coeffient_estimator_beta,adjusted_coefficient_of_determination,econometric_method
1,Corporate social responsibility financial instability and corporate financial performance: Linear non-linear and spillover effects – The case of the CAC 40 companies,Abderrahmane Jahmanea,Brahim Gaiesb,NA,2020,Journal Article,Finance Research Letters,3,3,...,NA,NA,0.95,Negative,NA,NA,NA,NA,NA,Correlation Matrix
1,Corporate social responsibility financial instability and corporate financial performance: Linear non-linear and spillover effects – The case of the CAC 40 companies,Abderrahmane Jahmanea,Brahim Gaiesb,NA,2020,Journal Article,Finance Research Letters,3,3,...,NA,NA,0.95,Negative,NA,NA,NA,NA,NA,Correlation Matrix


In [106]:
folder = 'Tables_0'
table_nb = 1
table = 'table_{}'.format(table_nb)
path = os.path.join(folder, table + '.txt')
if os.path.exists(folder) == False:
        os.mkdir(folder)
for ext in ['.txt', '.tex', '.pdf']:
    x = [a for a in os.listdir(folder) if a.endswith(ext)]
    [os.remove(os.path.join(folder, i)) for i in x]

In [50]:
table(df_final$sign_of_effect)


     Positive Insignificant      Negative 
          674           559           245 

In [96]:
df_final %>% 
group_by(sign_of_effect) %>%
summarize(
    mean(publication_year), median(publication_year),
mean(number_of_observations, na.rm=TRUE), median(number_of_observations, na.rm=TRUE))

sign_of_effect,mean(publication_year),median(publication_year),"mean(number_of_observations, na.rm = TRUE)","median(number_of_observations, na.rm = TRUE)"
Positive,2012.622,2015,2350.623,651.5
Insignificant,2012.581,2012,1662.771,494.0
Negative,2012.584,2013,3095.988,868.0


Not enough variability in the outcome

In [58]:
table(df_final$sign_of_effect, df_final$publication_type)

               
                ??? ADERSE Congress Journal Article Online Journal
  Positive        4               2             661              5
  Insignificant   5               4             537             12
  Negative        3               0             226             15
               
                Working Paper
  Positive                  2
  Insignificant             1
  Negative                  1

In [60]:
table(df_final$sign_of_effect, df_final$cnrs_ranking)

               
                  ?   0   1 1eg  1g   2   3   4
  Positive       20 273  34   9  95 118  75  50
  Insignificant  17 246  24   9  38 176  13  36
  Negative        3  90  15   2  21  72  21  21

Biased toward ` Environmental Social and Governance`

In [61]:
table(df_final$sign_of_effect, df_final$study_focused_on_social_environmental_behaviour)

               
                Environmental Environmental and Social Environmental Social
  Positive                 15                       58                  109
  Insignificant            15                       37                  101
  Negative                  6                        9                   57
               
                Environmental Social and Governance Social
  Positive                                      396     96
  Insignificant                                 378     28
  Negative                                      173      0

Not enough variability in the outcome: `???` and `Longitudinal study`

In [62]:
table(df_final$sign_of_effect, df_final$type_of_data)

               
                ??? Cross-section Longitudinal data Longitudinal study
  Positive        1           102                48                  7
  Insignificant   0           131                11                  8
  Negative        0            34                18                  0
               
                Panel data
  Positive             509
  Insignificant        407
  Negative             193

In [63]:
table(df_final$sign_of_effect, df_final$evaluation_method_of_the_link_between_csr_and_cfp)

               
                Correlation Regression
  Positive              289        278
  Insignificant         165        287
  Negative               85        120

In [64]:
table(df_final$sign_of_effect, df_final$developed_new)

               
                Developed Developing Worldwide
  Positive            278        120       162
  Insignificant       170         94       184
  Negative            101         40        64

Not enough variability in the outcome

In [65]:
table(df_final$sign_of_effect, df_final$econometric_method)

               
                2SLS  AR Correlation Matrix Fixed effects GLS GMM  IV
  Positive        18   6                290            47  10  17  11
  Insignificant    5   5                165            43   6  36   4
  Negative         4   3                 85            26   5  13   1
               
                Multiple Linear Regression Newey Non-linear
  Positive                              96     3          8
  Insignificant                        110     5         17
  Negative                              34     4          5
               
                Partial Least Squares Piecewise Pooled PSM Random Effects
  Positive                          7         6     10   3             15
  Insignificant                     1         4     30   0              6
  Negative                          0         5      5   0             13
               
                Simple Linear Regression Simultaneous model Structural Tobit
  Positive                             0   

The model controls for the paper's title but is not reported in the table

In [124]:
%get path table
t_0 <- multinom(sign_of_effect ~ 
                #number_of_observations + 
                publication_year + 
                #publication_type +
                study_focused_on_social_environmental_behaviour +
                evaluation_method_of_the_link_between_csr_and_cfp +
                developed_new + 
                nr,
                data=df_final)
            
dep <- "Dependent variable: Sign of effect"
#fe1 <- list(
#    c("XXXXX", "Yes")
#             )

#table_1 <- go_latex(list(
#    t_0#,t_1, t_2, t_3
#),
#    title="TITLE",
#    dep_var = dep,
#    addFE=FALSE,
#    save=TRUE,
#    note = FALSE,
#    name=path
#) 
stargazer(t_0, type="text",  omit = "nr", style = "qje")

# weights:  360 (238 variable)
initial  value 1332.616706 
iter  10 value 924.003806
iter  20 value 838.485661
iter  30 value 828.270992
iter  40 value 826.965945
iter  50 value 826.834577
iter  60 value 826.782601
iter  70 value 826.779133
final  value 826.778970 
converged

                                                                                       Insignificant         Negative     
                                                                                            (1)                 (2)       
--------------------------------------------------------------------------------------------------------------------------
publication_year                                                                         -0.004***           -0.006***    
                                                                                         (0.0001)            (0.0001)     
                                                                                                            

### relative risk ratios (marginal effect)

**How to read**

- Publication year
    - Keeping all other variables constant, if the publication increases one unit, the paper is 0.995 times more likely to stay in the negative category as compared to the positive category (the risk or odds is .05% lower).
    
When the RRR is above one, the odd of the positive class is stronger (larger probability, keeping everything else constant)    

In [125]:
t_0.rrr = exp(coef(t_0))
stargazer(t_0,
          type="text",
          coef=list(t_0.rrr),
          p.auto=FALSE,
          omit = "nr")


                                                                                       Dependent variable:     
                                                                                   ----------------------------
                                                                                   Insignificant    Negative   
                                                                                        (1)            (2)     
---------------------------------------------------------------------------------------------------------------
publication_year                                                                      0.996***      0.994***   
                                                                                      (0.0001)      (0.0001)   
                                                                                                               
study_focused_on_social_environmental_behaviourEnvironmental and Social               1.309**       9.0

In [92]:
#tbe1  = "Reference group: Positive " \
#"\sym{*} Significance at the 10\%, \sym{**} Significance at the 5\%, \sym{***} Significance at the 1\%."

#multicolumn ={
#    'Eligible': 2,
#    'Non-Eligible': 1,
#    'All': 1,
#    'All benchmark': 1,
#}

#multi_lines_dep = '(city/product/trade regime/year)'
#new_r = ['& Insignificant', 'Negative']
#lb.beautify(table_number = table_nb,
            #reorder_var = reorder,
            #multi_lines_dep = multi_lines_dep,
#            new_row= new_r,
            #multicolumn = multicolumn,
#            table_nte = tbe1,
#            jupyter_preview = True,
#            resolution = 200,
#            folder = folder)

# Generate reports

In [126]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp
import sys
path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)
sys.path.append(os.path.join(parent_path, 'utils'))
import make_toc

In [127]:
name_json = 'parameters_ETL_pollution_credit_constraint.json'
path_json = os.path.join(str(Path(path).parent.parent), 'utils',name_json)

In [128]:
def create_report(extension = "html", keep_code = False, notebookname = None):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            notebookname = notebookname  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    if keep_code:
        os.system('jupyter nbconvert --to {} {}'.format(
    extension,notebookname))
    else:
        os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [132]:
create_report(extension = "html", keep_code = True, notebookname = "00_replicate_tables.ipynb")

Report Available at this adress:
 /home/ubuntu/esg_metadata/02_data_analysis/01_model_estimation/00_replicate_results/Reports/00_replicate_tables.html


In [ ]:
### Update TOC in Github
for p in [parent_path,
          str(Path(path).parent),
          #os.path.join(str(Path(path).parent), "00_download_data_from"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "00_statistical_exploration"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "01_model_estimation"),
         ]:
    try:
        os.remove(os.path.join(p, 'README.md'))
    except:
        pass
    path_parameter = os.path.join(parent_path,'utils', name_json)
    md_lines =  make_toc.create_index(cwd = p, path_parameter = path_parameter)
    md_out_fn = os.path.join(p,'README.md')
    
    if p == parent_path:
    
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = True, path_parameter = path_parameter)
    else:
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = False)